In [5]:
from typing import Any, Dict, List, Optional, get_args, get_origin, get_type_hints, Union, Annotated
from pydantic import BaseModel, ValidationError, validator , model_validator, field_validator
from pydantic.fields import Field
from pydantic.functional_validators import AfterValidator
from graphviz import Digraph


In [6]:
file_url = r"C:\Users\Tommaso\Documents\Dev\Abstractions\data\conversations.json"
file_url = "/Users/tommasofurlanello/Documents/Dev/Abstractions/data/conversations.json"

In [7]:
import json

def load_first_record(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        if isinstance(data, list) and len(data) > 0:
            return data[0]  # Return the first item in the list
        else:
            raise ValueError("JSON is not a list or is empty")

def load_nth_record(file_path, n):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        if isinstance(data, list) and len(data) > n:
            return data[n]  # Return the nth item in the list
        else:
            raise ValueError("JSON is not a list or does not have enough items")

first_record = load_first_record(file_url)


In [8]:
first_record

{'title': 'Access Denied, Master',
 'create_time': 1710151351.377011,
 'update_time': 1710151386.243972,
 'mapping': {'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66': {'id': 'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66',
   'message': {'id': 'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66',
    'author': {'role': 'system', 'name': None, 'metadata': {}},
    'create_time': None,
    'update_time': None,
    'content': {'content_type': 'text', 'parts': ['']},
    'status': 'finished_successfully',
    'end_turn': True,
    'weight': 0.0,
    'metadata': {'is_visually_hidden_from_conversation': True},
    'recipient': 'all'},
   'parent': 'aaa135a9-3650-454d-9e77-4783d861c945',
   'children': ['aaa2b64e-48b5-424a-9d94-8a554fab4981']},
  'aaa135a9-3650-454d-9e77-4783d861c945': {'id': 'aaa135a9-3650-454d-9e77-4783d861c945',
   'message': None,
   'parent': None,
   'children': ['bad6f1ec-e065-46ce-b96a-3b0dd11b6e66']},
  'aaa2b64e-48b5-424a-9d94-8a554fab4981': {'id': 'aaa2b64e-48b5-424a-9d94-8a554fab4981',
   'messa

In [9]:
class Author(BaseModel):
    role: str
    name: Optional[str] = None
    metadata: Dict[str, Any] = {}

class Content(BaseModel):
    content_type: str
    parts: Optional[Union[List[Union[str, Dict[str, Any]]], Dict[str, Any]]] = None
    language: Optional[str] = None
    text: Optional[str] = None

    @field_validator('parts', mode= 'before')
    def validate_parts(cls, v):
        if isinstance(v, dict):
            return [v]  # Wrap in a list if it's a dict
        return v
    def get_message_content(self):
    # Check if text is provided
        content = self
        if content.text:
            return content.text
        # If text is None, try to get content from parts
        elif content.parts:
            # If parts is a list, we concatenate items or get the text from dict
            if isinstance(content.parts, list):
                # Concatenate list items if they are strings, or stringify them if they are dicts
                return ' '.join(str(part) if isinstance(part, dict) else part for part in content.parts)
            # If parts is a dict, we convert it to string
            elif isinstance(content.parts, dict):
                return str(content.parts)
        return 'No message'

class Message(BaseModel):
    id: str
    author: Optional[Author] = None
    create_time: Optional[float] = None
    update_time: Optional[float] = None
    content: Optional[Content] = None
    status: str
    end_turn: Optional[bool] = None
    weight: float
    metadata: Dict[str, Any] = {}
    recipient: str




In [10]:
from typing import List, Optional
from pydantic import BaseModel



class Node(BaseModel):
    id: str
    message: Optional[Message] = None
    parent: Optional[str] = None
    children: List[str] = []

    def is_leaf(self) -> bool:
        return len(self.children) == 0

    def is_root(self) -> bool:
        return self.parent is None

    def is_linear(self) -> bool:
        return len(self.children) <= 1 and (self.parent is None or self.parent.children == 1)

class Conversation(BaseModel):
    title: str
    create_time: float
    update_time: float
    mapping: Dict[str, Node]
    moderation_results: List[Any] = []
    current_node: str
    plugin_ids: Optional[Any] = None
    conversation_id: str
    conversation_template_id: Optional[Any] = None
    gizmo_id: Optional[Any] = None
    is_archived: bool
    safe_urls: List[Any] = []
    id: str

    def extract_all_linearized_conversations(self) -> List['LinearConversation']:
        def build_linear_conversations(node_id: str, current_path: Dict[str, Node], depth=0):
            node = self.mapping[node_id]
            new_node = node.model_copy(deep=True)
            new_node.children = new_node.children[:1]  # Keep only the first child to ensure linearity

            # Update the current path with the new linear node
            new_path = {**current_path, new_node.id: new_node}

            if not new_node.children:
                # Leaf node, store the current path as a linear conversation
                linear_conversations.append(LinearConversation(**{**self.model_dump(), "mapping": new_path}))
            else:
                # Recursively build the conversation for the next node in the path
                build_linear_conversations(new_node.children[0], new_path, depth + 1)

            # If the original node had multiple children, start new paths for each
            if len(node.children) > 1:
                for child_id in node.children[1:]:
                    build_linear_conversations(child_id, current_path, depth + 1)

        linear_conversations = []
        for node_id, node in self.mapping.items():
            if not node.parent:
                build_linear_conversations(node_id, {})

        return linear_conversations
    
    def visualize_conversation(self, filename: str = 'conversation_graph'):
        graph = Digraph(comment=self.title)

        def add_nodes_and_edges(node_id: str):
            node = self.mapping[node_id]
            if node.message and node.message.author:
                author_role = node.message.author.role  # User or Assistant
                # Use the get_message_content method to retrieve the message content
                message_content = node.message.content.get_message_content() if node.message.content else "No content"
                label = f"{author_role}: {message_content}"
            else:
                label = "No message"

            graph.node(node_id, label=label)

            for child_id in node.children:
                graph.edge(node_id, child_id)
                add_nodes_and_edges(child_id)

        for node_id, node in self.mapping.items():
            if node.is_root():
                add_nodes_and_edges(node_id)

        graph.render(filename, view=True, format='png')

    
def validate_single_child_node(node: Node) -> Node:
    if len(node.children) > 1:
        raise ValueError("Node has multiple children, not suitable for LinearConversation.")
    return node

NodeWithSingleChild = Annotated[Node, AfterValidator(validate_single_child_node)]

class LinearConversation(Conversation):
    mapping: Dict[str, NodeWithSingleChild]

    def __init__(self, **data: Any):
        super().__init__(**data)

In [11]:
first_record

{'title': 'Access Denied, Master',
 'create_time': 1710151351.377011,
 'update_time': 1710151386.243972,
 'mapping': {'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66': {'id': 'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66',
   'message': {'id': 'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66',
    'author': {'role': 'system', 'name': None, 'metadata': {}},
    'create_time': None,
    'update_time': None,
    'content': {'content_type': 'text', 'parts': ['']},
    'status': 'finished_successfully',
    'end_turn': True,
    'weight': 0.0,
    'metadata': {'is_visually_hidden_from_conversation': True},
    'recipient': 'all'},
   'parent': 'aaa135a9-3650-454d-9e77-4783d861c945',
   'children': ['aaa2b64e-48b5-424a-9d94-8a554fab4981']},
  'aaa135a9-3650-454d-9e77-4783d861c945': {'id': 'aaa135a9-3650-454d-9e77-4783d861c945',
   'message': None,
   'parent': None,
   'children': ['bad6f1ec-e065-46ce-b96a-3b0dd11b6e66']},
  'aaa2b64e-48b5-424a-9d94-8a554fab4981': {'id': 'aaa2b64e-48b5-424a-9d94-8a554fab4981',
   'messa

In [12]:
conversation_instance = Conversation(**first_record)
conversation_instance


Conversation(title='Access Denied, Master', create_time=1710151351.377011, update_time=1710151386.243972, mapping={'bad6f1ec-e065-46ce-b96a-3b0dd11b6e66': Node(id='bad6f1ec-e065-46ce-b96a-3b0dd11b6e66', message=Message(id='bad6f1ec-e065-46ce-b96a-3b0dd11b6e66', author=Author(role='system', name=None, metadata={}), create_time=None, update_time=None, content=Content(content_type='text', parts=[''], language=None, text=None), status='finished_successfully', end_turn=True, weight=0.0, metadata={'is_visually_hidden_from_conversation': True}, recipient='all'), parent='aaa135a9-3650-454d-9e77-4783d861c945', children=['aaa2b64e-48b5-424a-9d94-8a554fab4981']), 'aaa135a9-3650-454d-9e77-4783d861c945': Node(id='aaa135a9-3650-454d-9e77-4783d861c945', message=None, parent=None, children=['bad6f1ec-e065-46ce-b96a-3b0dd11b6e66']), 'aaa2b64e-48b5-424a-9d94-8a554fab4981': Node(id='aaa2b64e-48b5-424a-9d94-8a554fab4981', message=Message(id='aaa2b64e-48b5-424a-9d94-8a554fab4981', author=Author(role='user'

In [13]:
conversation_instance.visualize_conversation()

In [14]:
len(conversation_instance.extract_all_linearized_conversations())

1

In [15]:
test_record = load_nth_record(file_url, 3)

In [16]:
test_instance = Conversation(**test_record)
print(test_instance)
print(len(test_instance.extract_all_linearized_conversations()))

title='Functional Criteria for Compound Order' create_time=1709851858.42512 update_time=1709852062.470933 mapping={'dc35e96e-78eb-434f-ad69-2f84416bb49c': Node(id='dc35e96e-78eb-434f-ad69-2f84416bb49c', message=Message(id='dc35e96e-78eb-434f-ad69-2f84416bb49c', author=Author(role='system', name=None, metadata={}), create_time=None, update_time=None, content=Content(content_type='text', parts=[''], language=None, text=None), status='finished_successfully', end_turn=True, weight=0.0, metadata={'is_visually_hidden_from_conversation': True}, recipient='all'), parent='aaa15694-6345-43a7-be05-1c5b4f73f9f6', children=['aaa239c1-beeb-4101-be6a-3b95dc9b0466', 'aaa2529d-3ccd-4f97-8907-35cddef7bef1', 'aaa2d260-4979-4b5d-acd7-24d658ade059', 'aaa270c8-9410-41ff-a995-8ced8fa2891c']), 'aaa15694-6345-43a7-be05-1c5b4f73f9f6': Node(id='aaa15694-6345-43a7-be05-1c5b4f73f9f6', message=None, parent=None, children=['dc35e96e-78eb-434f-ad69-2f84416bb49c']), 'aaa239c1-beeb-4101-be6a-3b95dc9b0466': Node(id='aaa

In [17]:
test_instance.extract_all_linearized_conversations()

[LinearConversation(title='Functional Criteria for Compound Order', create_time=1709851858.42512, update_time=1709852062.470933, mapping={'aaa15694-6345-43a7-be05-1c5b4f73f9f6': Node(id='aaa15694-6345-43a7-be05-1c5b4f73f9f6', message=None, parent=None, children=['dc35e96e-78eb-434f-ad69-2f84416bb49c']), 'dc35e96e-78eb-434f-ad69-2f84416bb49c': Node(id='dc35e96e-78eb-434f-ad69-2f84416bb49c', message=Message(id='dc35e96e-78eb-434f-ad69-2f84416bb49c', author=Author(role='system', name=None, metadata={}), create_time=None, update_time=None, content=Content(content_type='text', parts=[''], language=None, text=None), status='finished_successfully', end_turn=True, weight=0.0, metadata={'is_visually_hidden_from_conversation': True}, recipient='all'), parent='aaa15694-6345-43a7-be05-1c5b4f73f9f6', children=['aaa239c1-beeb-4101-be6a-3b95dc9b0466']), 'aaa239c1-beeb-4101-be6a-3b95dc9b0466': Node(id='aaa239c1-beeb-4101-be6a-3b95dc9b0466', message=Message(id='aaa239c1-beeb-4101-be6a-3b95dc9b0466', au

In [18]:
test_instance.visualize_conversation()

In [19]:
import erdantic as erd

In [24]:
erd.draw(Conversation, out = 'gang.png')